In [1]:
import pandas as pd
import numpy as np
import requests
import re
from tqdm import tqdm
from time import sleep


In [ ]:
with open(r'd:\asn num with name.txt') as f:
    tt =f.readlines()

In [ ]:
tmp = [x for x in tt if 
       (x.find('University')>0 
        or x.find('College')>0 
        or x.find('School')>0
        or x.find('Polytechnic Institute')>0
        or x.find('Institute of Technology')>0) and x.endswith('US\n')]

In [ ]:
ANS = [t.split(' ')[0] for t in tmp]

In [ ]:
len(ANS)

In [ ]:
ANS[-10:]

In [ ]:
base = 'http://whois.arin.net/rest/asn/'
p = re.compile(r'<orgRef handle="(.*)" name="(.*)">(https://whois.arin.net/rest/org/.*)</orgRef>')
p2 = re.compile(r'(25[0-5]|2[0-4]\d|[0-1]\d{2}|[1-9]?\d)\.(25[0-5]|2[0-4]\d|[0-1]\d{2}|[1-9]?\d)\.(25[0-5]|2[0-4]\d|[0-1]\d{2}|[1-9]?\d)\.(25[0-5]|2[0-4]\d|[0-1]\d{2}|[1-9]?\d)')

#l = []
asn_list = [i[0] for i in l]

for asn in tqdm(ANS):
    if asn not in asn_list:
        # get the org handle, name, net_url use ans
        url1 = base + asn
        tmp = requests.get(url1)
        match = p.findall(tmp.text)
        handle,name,url2 = match[0]
        url2 = url2 + r'/nets'

        #get the net range use net_url
        tmp2 = requests.get(url2)
        match2 = p2.findall(tmp2.text)

        #re arrange the ip
        idx = range(0,len(match2),2)
        rg = [('.'.join(match2[i+1]),'.'.join(match2[i])) for i in idx]

        # merge the ANS, handle, name and range in a list
        l.append([asn,handle,name,rg])
        sleep(1)
        asn_list.append(asn)


In [ ]:
len(l),len(asn_list)

In [ ]:
l[-5:],asn_list[-5:]

In [ ]:
l[:10]

In [ ]:
df = pd.DataFrame(l,columns = ['asn','handle','name','range'])

In [ ]:
df.head(10)

In [ ]:
len(df.iloc[2,:]['range'])

In [ ]:
df[df['name'].isin(['Harvard University'])]

In [ ]:
df[(df.name.str.find('School District')>0)]

In [ ]:
# remove School District
df = df[~(df.name.str.find('School District')>0)]

In [ ]:
df.to_pickle(r'd:\asn handle name range for 1231 university')

In [ ]:
df.to_csv(r'd:\asn handle name range for 1231 university.csv')

In [ ]:
r_list = reduce(lambda x,y:x+y,df['range'].values);r_list[:20]

In [ ]:
r_set = set(r_list)

In [ ]:
len(r_list),len(r_set)

In [ ]:
def gen_ip3(ip_range):
    ip_list = []
    start,end = ip_range
    zone = zip(map(int,start.split('.')),map(int,end.split('.')))
    for i in range(zone[0][0],zone[0][1]+1):
        for j in range(zone[1][0],zone[1][1]+1):
            for k in range(zone[2][0],zone[2][1]+1):
                ip_list.append(".".join(map(str,[i,j,k])))
    return ip_list


In [2]:
def gen_ip4(ip_range):
    ip_list = []
    start,end = ip_range
    zone = zip(map(int,start.split('.')),map(int,end.split('.')))
    for i in range(zone[0][0],zone[0][1]+1):
        for j in range(zone[1][0],zone[1][1]+1):
            for k in range(zone[2][0],zone[2][1]+1):
                for l in range(zone[3][0],zone[3][1]+1):
                    ip_list.append(".".join(map(str,[i,j,k,l])))
    return ip_list

In [ ]:
## generate a full ip list which is 54,356,055 rows
ip4_list = []
for i in r_set:
    ip4_list += gen_ip4(i)

In [ ]:
with open(r'd:\4ip_list.txt','w') as f:
    for i in ip4_list:
        f.write(i+'\n')

In [ ]:
ip4_list[-1]

# this is for get org names and ip

In [3]:
df = pd.read_csv(r'd:\asn handle name range for 1231 university.csv');
df.head()

,Unnamed: 0,asn,handle,name,range
0,0,AS2,UNIVER-19,University of Delaware,"[(u'207.123.196.0', u'207.123.196.255'), (u'19..."
1,1,AS3,MIT-2,Massachusetts Institute of Technology,"[(u'128.30.0.0', u'128.30.255.255'), (u'128.31..."
2,2,AS4,USC-32,University of Southern California,[]
3,3,AS8,RICEUN,Rice University,"[(u'192.31.145.0', u'192.31.145.255'), (u'192...."
4,4,AS9,CARNEG-Z,Carnegie Mellon University,"[(u'192.58.107.0', u'192.58.107.255'), (u'192...."


In [4]:
name_list = df.name.unique();
name_list[:5]

array(['University of Delaware', 'Massachusetts Institute of Technology',
       'University of Southern California', 'Rice University',
       'Carnegie Mellon University'], dtype=object)

In [18]:
df_list = []
for i in name_list:
    for range_list in df[df.name == i].range.values:
        t = eval(range_list)
        for rg in t:
            ip_list = gen_ip4(rg)
            l = len(ip_list)
            tmp_df = pd.DataFrame(ip_list,index = [i]*l)
            df_list.append(tmp_df)
tmp_df.head()

,0
Collin County Community College District,192.231.40.0
Collin County Community College District,192.231.40.1
Collin County Community College District,192.231.40.2
Collin County Community College District,192.231.40.3
Collin County Community College District,192.231.40.4


In [19]:
pd.concat(df_list).to_csv(r'd:\new_org_ip.csv',header = False)

In [2]:
dff = pd.read_csv(r'd:\new_org_ip.csv')

In [3]:
dff.head()

,University of Delaware,207.123.196.0
0,University of Delaware,207.123.196.1
1,University of Delaware,207.123.196.2
2,University of Delaware,207.123.196.3
3,University of Delaware,207.123.196.4
4,University of Delaware,207.123.196.5


In [4]:
len(dff)

89757464

In [67]:
dff['207.123.196.0'].nunique()

52537138

In [33]:
df22 = dff.drop_duplicates(subset = '207.123.196.0')

In [65]:
df22.head()

,University of Delaware,207.123.196.0
0,University of Delaware,207.123.196.1
1,University of Delaware,207.123.196.2
2,University of Delaware,207.123.196.3
3,University of Delaware,207.123.196.4
4,University of Delaware,207.123.196.5


In [66]:
len(df22)

52537138

In [34]:
len(df22)

52537138

In [64]:
df22.to_csv(r'd:\org_ip.csv',index = False,header = False,sep = "|")

In [63]:
df22.head().to_csv(r'd:\test_org_ip.csv',index = False,header = False,sep = "|")